In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

os.chdir("..")

In [3]:
from pipeline.train import *
from pipeline.config import RUTA_MODELOS, RUTA_DATOS, RUTA_SP500

In [4]:
df = cargar_datos("S&P500", "2015-01-01", "2024-12-20")

In [30]:
modelo, metricas = entrenar_prophet(df)

NameError: name 'logger' is not defined

In [6]:
guardar_modelo(modelo, "S&P500", metricas)

In [7]:
from pipeline.inference import *

In [9]:
modelo_cargado =cargar_modelo("S&P500")

In [11]:
preds = realizar_prediccion(modelo_cargado, "2025-01-01", "2025-06-30")

In [12]:
visualizar_prediccion(preds, "S&P500", "2025-01-01", "2025-06-30", save_plot=True)

In [13]:
guardar_prediccion(preds, "S&P500", "2025-01-01", "2025-06-30")

In [14]:
import pandas as pd
pd.read_parquet("/Users/aladelca/Library/CloudStorage/GoogleDrive-alarcon.adrianc@gmail.com/My Drive/UPC - Sistemas/metodologia_data_science/repos/metodologia_para_data_science_20251/src/outputs/predicciones_GOOG_2025-01-01_2025-12-31_2025-05-17_11-40-31.parquet")

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2025-01-01,182.508361,171.266725,187.000698,182.508361,182.508361,-3.312720,-3.312720,-3.312720,1.266982,1.266982,1.266982,-4.579702,-4.579702,-4.579702,0.0,0.0,0.0,179.195642
1,2025-01-02,182.611575,172.119906,187.384162,182.611575,182.611575,-3.680596,-3.680596,-3.680596,1.207686,1.207686,1.207686,-4.888282,-4.888282,-4.888282,0.0,0.0,0.0,178.930979
2,2025-01-03,182.714789,171.536429,186.314973,182.714789,182.714789,-3.808808,-3.808808,-3.808808,1.379139,1.379139,1.379139,-5.187947,-5.187947,-5.187947,0.0,0.0,0.0,178.905981
3,2025-01-06,183.024430,170.461648,185.785004,183.024430,183.024430,-4.672996,-4.672996,-4.672996,1.297000,1.297000,1.297000,-5.969996,-5.969996,-5.969996,0.0,0.0,0.0,178.351434
4,2025-01-07,183.127644,170.316018,185.737322,183.127644,183.127644,-4.873214,-4.873214,-4.873214,1.298412,1.298412,1.298412,-6.171627,-6.171627,-6.171627,0.0,0.0,0.0,178.254430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,2025-12-25,219.458914,183.751615,248.393504,186.241151,248.619635,-1.413510,-1.413510,-1.413510,1.207686,1.207686,1.207686,-2.621197,-2.621197,-2.621197,0.0,0.0,0.0,218.045404
257,2025-12-26,219.562128,182.974655,249.908451,186.168536,248.923826,-1.444942,-1.444942,-1.444942,1.379139,1.379139,1.379139,-2.824081,-2.824081,-2.824081,0.0,0.0,0.0,218.117186
258,2025-12-29,219.871770,182.741173,248.565302,186.302349,249.379299,-2.293987,-2.293987,-2.293987,1.297000,1.297000,1.297000,-3.590986,-3.590986,-3.590986,0.0,0.0,0.0,217.577783
259,2025-12-30,219.974983,181.895820,248.714476,186.229734,249.524270,-2.587152,-2.587152,-2.587152,1.298412,1.298412,1.298412,-3.885564,-3.885564,-3.885564,0.0,0.0,0.0,217.387831


In [4]:
"""Pruebas para el módulo de entrenamiento."""
import pytest
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
import shutil

from pipeline.train import cargar_datos, entrenar_prophet, guardar_modelo
from pipeline.config import RUTA_MODELOS



def datos_prueba():
    """Fixture para crear datos de prueba."""
    # Crear fechas
    fechas = pd.date_range(start='2020-01-01', end='2020-12-31', freq='D')
    
    # Crear datos sintéticos
    np.random.seed(42)
    precios = np.random.normal(100, 10, len(fechas)).cumsum()
    
    # Crear DataFrame
    df = pd.DataFrame({
        'fecha': fechas,
        'precio_cierre': precios
    })
    
    return df



def directorio_temporal():
    """Fixture para crear un directorio temporal para pruebas."""
    # Crear directorio temporal
    temp_dir = 'temp_test'
    os.makedirs(temp_dir, exist_ok=True)
    
    # Guardar ruta original
    ruta_original = RUTA_MODELOS
    
    # Modificar ruta para pruebas
    import src.pipeline.config
    src.pipeline.config.RUTA_MODELOS = temp_dir
    
    yield temp_dir
    
    # Restaurar ruta original
    src.pipeline.config.RUTA_MODELOS = ruta_original
    
    # Limpiar después de las pruebas
    shutil.rmtree(temp_dir)


def test_cargar_datos(datos_prueba, tmp_path):
    """Prueba la función cargar_datos."""
    # Guardar datos de prueba
    ruta_archivo = tmp_path / "AAPL.csv"
    datos_prueba.to_csv(ruta_archivo, index=False)
    
    # Probar carga de datos
    df = cargar_datos(
        "AAPL",
        "2020-01-01",
        "2020-12-31"
    )
    
    assert isinstance(df, pd.DataFrame)
    assert 'ds' in df.columns
    assert 'y' in df.columns
    assert len(df) > 0


def test_cargar_datos_fecha_invalida(datos_prueba, tmp_path):
    """Prueba la función cargar_datos con fechas inválidas."""
    # Guardar datos de prueba
    ruta_archivo = tmp_path / "AAPL.csv"
    datos_prueba.to_csv(ruta_archivo, index=False)
    
    # Probar con fecha de inicio posterior a fecha de corte
    with pytest.raises(ValueError):
        cargar_datos(
            "AAPL",
            "2020-12-31",
            "2020-01-01"
        )


def test_entrenar_prophet(datos_prueba):
    """Prueba la función entrenar_prophet."""
    # Preparar datos
    df = datos_prueba.rename(columns={
        'fecha': 'ds',
        'precio_cierre': 'y'
    })
    
    # Entrenar modelo
    modelo, metricas = entrenar_prophet(
        df,
        "AAPL",
        estacionalidad_anual=True,
        estacionalidad_semanal=True,
        estacionalidad_diaria=False,
        cambio_punto=0.05
    )
    
    assert modelo is not None
    assert isinstance(metricas, dict)
    assert 'mse' in metricas
    assert 'rmse' in metricas
    assert 'mae' in metricas
    assert 'r2' in metricas


def test_guardar_modelo(datos_prueba, directorio_temporal):
    """Prueba la función guardar_modelo."""
    # Preparar datos y entrenar modelo
    df = datos_prueba.rename(columns={
        'fecha': 'ds',
        'precio_cierre': 'y'
    })
    
    modelo, metricas = entrenar_prophet(
        df,
        "AAPL",
        estacionalidad_anual=True,
        estacionalidad_semanal=True,
        estacionalidad_diaria=False,
        cambio_punto=0.05
    )
    
    # Guardar modelo
    guardar_modelo(modelo, "AAPL", metricas, directorio_temporal)
    
    # Verificar que se crearon los archivos
    assert os.path.exists(os.path.join(directorio_temporal, "prophet_AAPL.joblib"))
    assert os.path.exists(os.path.join(directorio_temporal, "metricas_AAPL.json")) 

In [5]:
datos_prueba()

,fecha,precio_cierre
0,2020-01-01,104.967142
1,2020-01-02,203.584499
2,2020-01-03,310.061384
3,2020-01-04,425.291682
4,2020-01-05,522.950149
...,...,...
361,2020-12-27,36226.473412
362,2020-12-28,36325.385811
363,2020-12-29,36429.402928
364,2020-12-30,36536.304368


In [7]:
df = datos_prueba().rename(columns={
        'fecha': 'ds',
        'precio_cierre': 'y'
    })
    
modelo, metricas = entrenar_prophet(
    df = df
)

# Guardar modelo
guardar_modelo(modelo, "AAPL", metricas, directorio_temporal())

# Verificar que se crearon los archivos
assert os.path.exists(os.path.join(directorio_temporal(), "prophet_AAPL.joblib"))
assert os.path.exists(os.path.join(directorio_temporal(), "metricas_AAPL.json")) 

2025-05-17 12:09:26,481 - pipeline.train - INFO - Iniciando entrenamiento del modelo


2025-05-17 12:09:26,508 - pipeline.train - INFO - Ajustando modelo...
2025-05-17 12:09:26,519 - cmdstanpy - DEBUG - input tempfile: /var/folders/3s/3k3nlxhn13nfv9vncl9g9kbr0000gn/T/tmpy2r2f3hv/umyk9jpz.json
2025-05-17 12:09:26,528 - cmdstanpy - DEBUG - input tempfile: /var/folders/3s/3k3nlxhn13nfv9vncl9g9kbr0000gn/T/tmpy2r2f3hv/88arxr39.json
2025-05-17 12:09:26,529 - cmdstanpy - DEBUG - idx 0
2025-05-17 12:09:26,529 - cmdstanpy - DEBUG - running CmdStan, num_threads: None
2025-05-17 12:09:26,530 - cmdstanpy - DEBUG - CmdStan args: ['/Users/aladelca/Library/CloudStorage/GoogleDrive-alarcon.adrianc@gmail.com/My Drive/UPC - Sistemas/metodologia_data_science/repos/metodologia_para_data_science_20251/.venv/lib/python3.12/site-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=87466', 'data', 'file=/var/folders/3s/3k3nlxhn13nfv9vncl9g9kbr0000gn/T/tmpy2r2f3hv/umyk9jpz.json', 'init=/var/folders/3s/3k3nlxhn13nfv9vncl9g9kbr0000gn/T/tmpy2r2f3hv/88arxr39.json', 'output', 'file=/var/fo

12:09:26 - cmdstanpy - INFO - Chain [1] start processing


2025-05-17 12:09:26,530 - cmdstanpy - INFO - Chain [1] start processing


12:09:26 - cmdstanpy - INFO - Chain [1] done processing


2025-05-17 12:09:26,646 - cmdstanpy - INFO - Chain [1] done processing
2025-05-17 12:09:26,649 - pipeline.train - INFO - Calculando métricas...
2025-05-17 12:09:26,698 - pipeline.train - INFO - Métricas calculadas: {'mse': 84.94411703099702, 'rmse': 9.216513279488996, 'mae': 7.3958501535817796, 'r2': 0.9999992436002502}
2025-05-17 12:09:26,700 - pipeline.train - INFO - Guardando modelo y métricas para AAPL
2025-05-17 12:09:26,707 - pipeline.train - INFO - Modelo y métricas guardados exitosamente


TypeError: expected str, bytes or os.PathLike object, not generator

In [8]:
directorio_temporal()

<generator object directorio_temporal at 0x323086c00>

In [ ]:
directorio_temporal()

In [29]:
modelo, metricas = entrenar_prophet(
    df = df,
    tick="AAPL"
)

TypeError: entrenar_prophet() got an unexpected keyword argument 'tick'

In [12]:
cargar_datos(
            "TSLA",
            "2020-12-31",
            "2020-01-01"
        )

2025-05-17 12:22:18,050 - pipeline.train - INFO - Cargando datos para TSLA desde 2020-12-31 hasta 2020-01-01
2025-05-17 12:22:18,593 - pipeline.train - INFO - Datos cargados: 0 registros


,Date,Symbol,y,ds
